# Coding Challenge Huk-Coburg

In [ ]:
import mlflow
import sklearn
import pandas as pd

mlflow.sklearn.autolog()
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("ClaimsPrediction")

## Data Preparation - Feature Engineering

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_pickle("data/df_full_1.pkl")

In [ ]:
# ClaimNb, ClaimAmount and exposure are dependent variables of ClaimNorm 
df = df.drop(["ClaimNb", "ClaimAmount", "Exposure"], axis=1)

In [ ]:
labels = df["ClaimNorm"].to_numpy()
data = df.drop("ClaimNorm", axis=1)

### Train Test Split

In [ ]:
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.1, random_state=0)

#### Is the split unbiased (similar ratio of no-claims)?

In [ ]:
print("Ratio no-claims in Test", sum(labels_test > 0)/len(labels_test))

In [ ]:
print("Ratio no-claims in Train", sum(labels_train > 0)/len(labels_train))

## Metrics
The standard metric for a regression task is R2 the coefficient of determination. It is the standard metric for regressors in sklearn. 

## Baseline - Linear Model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
with mlflow.start_run(run_name="Baseline"):
    linear_model = LinearRegression()
    linear_model.fit(data_train, labels_train)
    linear_model.score(data_test, labels_test)

## Hyperparameter Tuning - Ridge Regression

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

from sklearn.linear_model import Ridge
from scipy.stats import uniform

ridge = Ridge()
distributions = dict(tol=[1e-4, 5e-4, 1e-3, 5e-3],
                     alpha=uniform(loc=0, scale=5))

with mlflow.start_run(run_name="HT-LinearModel"):
    reg = RandomizedSearchCV(ridge, distributions, random_state=0)
    search = reg.fit(data_train, labels_train)

In [ ]:
search.score(data_test, labels_test)

## Hyperparameter Tuning - Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
distributions = dict(n_estimators=[10,50,100],
                     max_depth=[2,3,4,5])

with mlflow.start_run(run_name="HT-RandomForest"):
    reg = RandomizedSearchCV(rf, distributions, random_state=0)
    search = reg.fit(data_train, labels_train)
    search.score(data_test, labels_test)